# CSV Processor Pro: Data Analysis Dashboard
This notebook provides an interactive interface to analyze and visualize your CSV data.

--- 
### ⚠️ 1. SETUP: PATH RESOLUTION
Run this cell first! It tells Python where your scripts are located.

In [ ]:
import os
import sys
from pathlib import Path

# Locate the project root (one level up from this notebook)
current_dir = Path(os.getcwd())
if current_dir.name == 'notebooks':
    project_root = current_dir.parent
else:
    project_root = current_dir

project_root_str = str(project_root.resolve())

# Add the project root to Python's search path
if project_root_str not in sys.path:
    sys.path.insert(0, project_root_str)

print(f"🔍 Scanning for scripts in: {project_root_str}")

try:
    from scripts.kpi_analyzer import calculate_kpis
    from scripts.visualizer import generate_visualizations
    print("✅ SUCCESS: 'scripts' module found and loaded!")
except ImportError as e:
    print(f"❌ ERROR: Still cannot find scripts. Error: {e}")
    print(f"Check: Does {os.path.join(project_root_str, 'scripts')} exist?")

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load data
data_path = os.path.join(project_root_str, 'data', 'large_sample.csv')
if os.path.exists(data_path):
    df = pd.read_csv(data_path)
    print(f"✅ SUCCESS: Loaded {len(df)} rows from {data_path}")
else:
    print(f"❌ ERROR: Data file not found at {data_path}")

### 📊 2. Key Performance Indicators (KPIs)

In [ ]:
if 'df' in locals():
    kpis = calculate_kpis(df)
    print("--- Summary KPIs ---")
    for k, v in kpis.items():
        print(f"{k.replace('_', ' ').title()}: {v:,.2f}" if isinstance(v, float) else f"{k.replace('_', ' ').title()}: {v}")

### 📈 3. Visualizations

In [ ]:
if 'df' in locals():
    sns.set_theme(style="whitegrid")
    plt.figure(figsize=(10, 6))
    sns.histplot(df['salary'], kde=True, color='green')
    plt.title('Overall Salary Distribution')
    plt.show()